<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [2]:
def get_data_and_first_cleaning():
    rooth_path = '../rawdata/'
    failures_df = pd.read_csv(rooth_path + 'wind-farm-1-failures-training.csv', sep=';')
    locations_df = pd.read_csv(rooth_path + "wind-farm-1-locations.csv", sep=';')
    logs_df = pd.read_csv(rooth_path + 'wind-farm-1-logs-training.csv', sep=';')
    metmast_df = pd.read_csv(rooth_path + 'wind-farm-1-metmast-training.csv', sep=';')
    signals_df = pd.read_csv(rooth_path + 'wind-farm-1-signals-training.csv', sep=';')
    power_df = pd.read_csv(rooth_path + 'Power_curve.csv', sep=',')
#     costs_df = pd.read_csv(rooth_path + 'HTW_Costs.csv')
    
    # pass all the Timestamps to date-time format
    failures_df['Timestamp'] = pd.to_datetime(failures_df['Timestamp'])
    logs_df['Timestamp'] = pd.to_datetime(logs_df['TimeDetected'])
    logs_df = logs_df.drop(columns=['TimeDetected','UnitTitleDestination'])
    metmast_df['Timestamp'] = pd.to_datetime(metmast_df['Timestamp'])
    signals_df['Timestamp'] = pd.to_datetime(signals_df['Timestamp'])
     
    # Drop Columns with std=0 by DA
    drop_features_metmast = ['Anemometer1_Offset', 'Anemometer2_Freq', 'Anemometer2_Offset', 'Min_Raindetection',
                             'Avg_Raindetection', 'Anemometer1_CorrGain', 'Anemometer1_CorrOffset',
                             'Anemometer2_CorrGain', 'Anemometer2_CorrOffset', 'DistanceAirPress', 
                             'AirRessureSensorZeroOffset']
    metmast_df = metmast_df.drop(columns=drop_features_metmast)
    
    drop_features_signals = ['Prod_LatestAvg_ActPwrGen2', 'Prod_LatestAvg_ReactPwrGen2']
    signals_df = signals_df.drop(columns=drop_features_signals)
    
    failures_df = failures_df.drop(columns='Remarks')
    
    #Fill na by interpolate
    signals_df = signals_df.bfill()
    
    return failures_df, locations_df, logs_df, metmast_df, signals_df, power_df

In [3]:
failures_df, locations_df, logs_df, metmast_df, signals_df, power_df = get_data_and_first_cleaning()

In [4]:
failures_df.head()

,Turbine_ID,Component,Timestamp
0,T11,GENERATOR,2016-03-03 19:00:00+00:00
1,T06,HYDRAULIC_GROUP,2016-04-04 18:53:00+00:00
2,T07,GENERATOR_BEARING,2016-04-30 12:40:00+00:00
3,T09,GENERATOR_BEARING,2016-06-07 16:59:00+00:00
4,T07,TRANSFORMER,2016-07-10 03:46:00+00:00


In [5]:
# Function to find str in columns of df
def component(component, col):
    pair_comp_col=[]
    for i in col:
        if component in i:
            pair_comp_col.append(i)
    return pair_comp_col

In [6]:
# Classification label for failures
failures_generator = failures_df[failures_df['Component'] == 'GENERATOR']
failures_generator.replace('GENERATOR', 1, inplace=True)
failures_gen_bear = failures_df[failures_df['Component'] == 'GENERATOR_BEARING']
failures_gen_bear.replace('GENERATOR_BEARING', 1, inplace=True)
failures_hyd = failures_df[failures_df['Component'] == 'HYDRAULIC_GROUP']
failures_hyd.replace('HYDRAULIC_GROUP', 1, inplace=True)
failures_gearbox = failures_df[failures_df['Component'] == 'GEARBOX']
failures_gearbox.replace('GEARBOX', 1, inplace=True)
failures_transf = failures_df[failures_df['Component'] == 'TRANSFORMER']
failures_transf.replace('TRANSFORMER', 1, inplace=True)

c:\users\ricar\.venvs\lewagon\lib\site-packages\pandas\core\frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [28]:
failures_df.sort_values(['Turbine_ID', 'Component'])

,Turbine_ID,Component,Timestamp
6,T01,GEARBOX,2016-07-18 02:10:00+00:00
19,T01,TRANSFORMER,2017-08-11 13:14:00+00:00
5,T06,GENERATOR,2016-07-11 19:48:00+00:00
7,T06,GENERATOR,2016-07-24 17:01:00+00:00
10,T06,GENERATOR,2016-09-04 08:08:00+00:00
11,T06,GENERATOR,2016-10-02 17:08:00+00:00
15,T06,GENERATOR,2016-10-27 16:26:00+00:00
1,T06,HYDRAULIC_GROUP,2016-04-04 18:53:00+00:00
20,T06,HYDRAULIC_GROUP,2017-08-19 09:47:00+00:00
22,T07,GENERATOR,2017-08-21 14:47:00+00:00


In [7]:
#Features to create each DF
def create_features(df):
    time_id = ['Timestamp', 'Turbine_ID']
    pair_hyd = component('Hyd', df.columns)
    pair_trafo = component('Trafo', df.columns)
    pair_gear = component('Gear', df.columns)
    pair_gen = component('Gen', df.columns)
    pair_rot = component('Rtr', df.columns)
    pair_amb = component('Amb', df.columns)
    pair_blds = component('Blds', df.columns)
    pair_cont = component('Cont', df.columns)
    pair_nac = component('Nac', df.columns)
    pair_spin = component('Spin', df.columns)
    pair_bus = component('Busbar', df.columns)
    pair_vol = component('Volt', df.columns)
    
    #Create DF for each component
    df_generator = df[time_id + pair_gen + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_hyd]
    df_gen_bear = df[time_id + pair_gen + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_hyd]
    df_transformer = df[time_id + pair_trafo + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_vol] 
    df_hydraulic = df[time_id + pair_hyd + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus] 
    df_gearbox = df[time_id + pair_gear + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_hyd]
    
    return df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox

In [8]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = create_features(signals_df)

In [9]:
# Merge DF Failures with DF of each component to know the moment of each failure
generator_final = df_generator.merge(failures_generator, on=['Timestamp','Turbine_ID'], how='outer')
gen_bear_final = df_gen_bear.merge(failures_gen_bear, on=['Timestamp', 'Turbine_ID'], how='outer')
hyd_final = df_hydraulic.merge(failures_hyd, on=['Timestamp', 'Turbine_ID'], how='outer')
gearbox_final = df_gearbox.merge(failures_gearbox, on=['Timestamp', 'Turbine_ID'], how='outer')
transf_final = df_transformer.merge(failures_transf, on=['Timestamp', 'Turbine_ID'], how='outer')

In [10]:
#Fill na on component column
generator_final['Component'].fillna(0, inplace=True)
gen_bear_final['Component'].fillna(0, inplace=True)
hyd_final['Component'].fillna(0, inplace=True)
gearbox_final['Component'].fillna(0, inplace=True)
transf_final['Component'].fillna(0, inplace=True)

In [11]:
# Set Timestamp as Index
generator_final.set_index('Timestamp', inplace=True)
gen_bear_final.set_index('Timestamp', inplace=True)
hyd_final.set_index('Timestamp', inplace=True)
gearbox_final.set_index('Timestamp', inplace=True)
transf_final.set_index('Timestamp', inplace=True)

In [12]:
# Sort Index
generator_final.sort_index(inplace=True)
gen_bear_final.sort_index(inplace=True)                                      
hyd_final.sort_index(inplace=True)
gearbox_final.sort_index(inplace=True)
transf_final.sort_index(inplace=True)

In [13]:
# Reset Index to group by Timestamp and Turbine_ID
generator_final.reset_index(inplace=True)
gen_bear_final.reset_index(inplace=True)                                      
hyd_final.reset_index(inplace=True)
gearbox_final.reset_index(inplace=True)
transf_final.reset_index(inplace=True)

In [14]:
def prepare_data(df, strategy='mean'):
    if strategy == 'mean':
        df = df.groupby(by=['Turbine_ID','Timestamp']).mean()
    else:
        df = df.grouby(by=['Turbine_ID', 'Timestamp']).max()
    
    # Fill na by interpolate
    df = df.bfill().reset_index()
        
    return df

In [29]:
df_generator_final=prepare_data(generator_final)
df_generator_final
# df_gen_bear_final=prepare_data(gen_bear_final)
# df_hyd_final=prepare_data(hyd_final)
# df_gearbox_final=prepare_data(gearbox_final)
# df_transf_final=prepare_data(transf_final)

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Cont_Hub_Temp_Avg,Cont_VCP_Temp_Avg,Cont_VCP_ChokcoilTemp_Avg,Cont_VCP_WtrTemp_Avg,Nac_Temp_Avg,Nac_Direction_Avg,Spin_Temp_Avg,Grd_Busbar_Temp_Avg,Hyd_Oil_Temp_Avg,Component
0,T01,2016-01-01 00:00:00+00:00,1277.4,1226.1,1249.0,9.0,41.0,58.0,59.0,58.0,...,28.0,43.0,91.0,39.0,28.0,218.5,20.0,38.0,30.0,0.0
1,T01,2016-01-01 00:10:00+00:00,1268.3,93.3,999.7,435.9,41.0,58.0,59.0,59.0,...,28.0,42.0,91.0,37.0,28.0,218.5,20.0,38.0,30.0,0.0
2,T01,2016-01-01 00:20:00+00:00,1394.4,220.3,774.0,486.1,41.0,57.0,57.0,57.0,...,28.0,42.0,87.0,38.0,29.0,213.3,20.0,38.0,30.0,0.0
3,T01,2016-01-01 00:30:00+00:00,1306.6,1212.4,1257.1,17.0,40.0,56.0,57.0,57.0,...,28.0,42.0,87.0,38.0,28.0,222.4,20.0,38.0,30.0,0.0
4,T01,2016-01-01 00:40:00+00:00,1320.9,1224.1,1257.7,18.0,40.0,57.0,58.0,57.0,...,28.0,42.0,88.0,39.0,28.0,222.4,20.0,38.0,30.0,0.0
5,T01,2016-01-01 00:50:00+00:00,1462.6,1249.5,1328.5,48.3,40.0,58.0,59.0,58.0,...,28.0,42.0,89.0,38.0,28.0,221.3,20.0,38.0,30.0,0.0
6,T01,2016-01-01 01:00:00+00:00,1350.0,1228.0,1265.5,22.5,40.0,59.0,60.0,59.0,...,28.0,42.0,90.0,39.0,28.0,221.3,20.0,38.0,30.0,0.0
7,T01,2016-01-01 01:10:00+00:00,1335.2,1225.1,1268.7,18.6,40.0,59.0,60.0,60.0,...,28.0,42.0,91.0,38.0,28.0,221.3,20.0,38.0,30.0,0.0
8,T01,2016-01-01 01:20:00+00:00,1370.7,1230.8,1279.4,25.6,40.0,60.0,61.0,60.0,...,28.0,43.0,92.0,38.0,28.0,221.3,20.0,39.0,30.0,0.0
9,T01,2016-01-01 01:30:00+00:00,1375.7,1212.2,1272.0,33.5,41.0,61.0,61.0,61.0,...,28.0,43.0,93.0,38.0,28.0,221.3,20.0,39.0,30.0,0.0


In [16]:
print(df_generator_final.isnull().values.any())
print(df_gen_bear_final.isnull().values.any())
print(df_hyd_final.isnull().values.any())
print(df_gearbox_final.isnull().values.any())
print(df_transf_final.isnull().values.any())

False
False
False
False
False


In [17]:
def fill_na_by_turbine(df):
    turbines = ['T01', 'T06', 'T07', 'T09', 'T11']
    df_ = pd.DataFrame(columns=df.columns)
    for turbine in turbines:
        df1 = df.loc[df['Turbine_ID']==turbine]
        if df1['Component'].nunique()>1:
            index = df1[df1['Component']==1]
            index['date'] = index['Timestamp']
            index = index[['date','Timestamp', 'Turbine_ID']]
            df_merged = df1.merge(index, how='left', on=['Turbine_ID','Timestamp'])
            df_merged = df_merged.fillna(method='bfill')
            
            #If there is not a failure after, hold present date
            df_merged['date'] = df_merged['date'].fillna(df_merged['Timestamp'])
            
            df_merged['TTF'] = round((df_merged['date'] - 
                                      df_merged['Timestamp']) / np.timedelta64(1, 'D'),0)
        else:
            df_merged = df1
            df_merged['date'] = df_merged['Timestamp']
#             df1 = df1[['date','Timestamp', 'Turbine_ID']]
#             df_merged = df.merge(df1, how='left', on=['Turbine_ID','Timestamp'])
            
#             df_merged['date'] = df_merged['date'].fillna(df_merged['Timestamp'])
            df_merged['TTF'] = 0 # df_merged['date'] - df_merged['Timestamp'] 

        #Drop Column Date
        df_final = df_merged.drop(columns='date')

        #df_final['TTF'] = df_final['TTF'].dt.days

        df_ = pd.concat([df_, df_final])

    return df_
        


In [18]:
generator_df = fill_na_by_turbine(df_generator_final)
gen_bear_df = fill_na_by_turbine(df_gen_bear_final)
hyd_df = fill_na_by_turbine(df_hyd_final)
gearbox_df = fill_na_by_turbine(df_gearbox_final)
transf_df = fill_na_by_turbine(df_transf_final)

c:\users\ricar\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\ricar\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\ricar\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warn

KeyboardInterrupt: 

In [ ]:
def Failure_Time_Horizon(days, period):
    if 2 <= days <= period:
        Flag=1
    else:
        Flag=0
    return Flag

In [ ]:
generator_df.dtypes

In [21]:
generator_df['60_days'] = generator_df.apply(lambda x: Failure_Time_Horizon(x['TTF'], 60),axis=1)
gen_bear_df['60_days'] = gen_bear_df.apply(lambda x: Failure_Time_Horizon(x['TTF'], 60),axis=1)
hyd_df['60_days'] = hyd_df.apply(lambda x: Failure_Time_Horizon(x['TTF'], 60),axis=1)
gearbox_df['60_days'] = gearbox_df.apply(lambda x: Failure_Time_Horizon(x['TTF'], 60),axis=1)
transf_df['60_days'] = transf_df.apply(lambda x: Failure_Time_Horizon(x['TTF'], 60),axis=1)

In [27]:
generator_df

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Cont_VCP_ChokcoilTemp_Avg,Cont_VCP_WtrTemp_Avg,Nac_Temp_Avg,Nac_Direction_Avg,Spin_Temp_Avg,Grd_Busbar_Temp_Avg,Hyd_Oil_Temp_Avg,Component,TTF,60_days
0,T01,2016-01-01 00:00:00+00:00,1277.4,1226.1,1249.0,9.0,41.0,58.0,59.0,58.0,...,91.0,39.0,28.0,218.5,20.0,38.0,30.0,0.0,0.0,0
1,T01,2016-01-01 00:10:00+00:00,1268.3,93.3,999.7,435.9,41.0,58.0,59.0,59.0,...,91.0,37.0,28.0,218.5,20.0,38.0,30.0,0.0,0.0,0
2,T01,2016-01-01 00:20:00+00:00,1394.4,220.3,774.0,486.1,41.0,57.0,57.0,57.0,...,87.0,38.0,29.0,213.3,20.0,38.0,30.0,0.0,0.0,0
3,T01,2016-01-01 00:30:00+00:00,1306.6,1212.4,1257.1,17.0,40.0,56.0,57.0,57.0,...,87.0,38.0,28.0,222.4,20.0,38.0,30.0,0.0,0.0,0
4,T01,2016-01-01 00:40:00+00:00,1320.9,1224.1,1257.7,18.0,40.0,57.0,58.0,57.0,...,88.0,39.0,28.0,222.4,20.0,38.0,30.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434117,T11,2017-08-31 23:20:00+00:00,1434.3,1243.1,1286.1,46.8,43.0,56.0,57.0,56.0,...,70.0,40.0,31.0,117.5,28.0,40.0,40.0,0.0,0.0,0
434118,T11,2017-08-31 23:30:00+00:00,1265.2,1238.6,1249.8,4.1,43.0,57.0,58.0,58.0,...,73.0,40.0,31.0,117.5,28.0,41.0,40.0,0.0,0.0,0
434119,T11,2017-08-31 23:40:00+00:00,1264.6,1235.1,1248.8,4.7,44.0,58.0,59.0,59.0,...,76.0,39.0,31.0,116.0,28.0,41.0,40.0,0.0,0.0,0
434120,T11,2017-08-31 23:50:00+00:00,1281.4,1231.9,1250.5,6.3,45.0,59.0,60.0,60.0,...,78.0,39.0,31.0,102.9,28.0,41.0,40.0,0.0,0.0,0


In [22]:
# def timestamp_round_down(df, time_column='Timestamp'):
#     'Arredondar os intervalos de tempo para os 10 minutos anteriores'
#     df[time_column] = df.apply(lambda x: x[time_column] - datetime.timedelta(minutes=x[time_column].minute % -10,seconds=x[time_column].second, microseconds=x[time_column].microsecond),axis=1)
#     return df

In [23]:
# def time_transform(df, time_column='Timestamp'):
#     'Transformar as colunas referentes a tempo no data typw tempo'
# #     df[time_column] = pd.to_datetime(df[time_column].str[:19])
#     # df[time_column] = df[time_column].dt.tz_localize('GMT')
#     df[time_column] = df[time_column].dt.tz_convert(None)
#     return df